In [1]:
from pymongo import MongoClient
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from datetime import datetime
import os 
from dotenv import load_dotenv
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

load_dotenv()
# MongoDB Setup
# Connect to MongoDB (replace the connection string with your MongoDB credentials)
client = MongoClient(os.getenv("MONGO_URI"))  # Local MongoDB connection
db = client[os.getenv("DB")]
temperature_collection = db[os.getenv("COLLECTION")]

/home/mori/miniconda3/envs/llms/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/mori/miniconda3/envs/llms/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
def get_occupancy(time):

    # Adjust date to the specific time (23:54:48)
    
    # Convert datetime to string in the required format
    query_time_str = time.strftime("%Y-%m-%dT%H:%M:%S")
    
    # Perform the query using the formatted string
    record = temperature_collection.find_one({"time": query_time_str})
    
    if record:
        return record.get('Temp', "No temperature data available")
    else:
        print("No record found")
        return "No temperature data available for 2 PM on this date."

In [3]:
# Load a local model (using GPT-2 here, you can replace with a larger model if needed)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = os.getenv("MODEL"), # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = HF_TOKEN
    )
FastLanguageModel.for_inference(model) 

local_llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

/home/mori/miniconda3/envs/llms/lib/python3.10/site-packages/pymongo/ocsp_support.py:284: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to next_update_utc.
  if response.next_update and response.next_update < now:
/home/mori/miniconda3/envs/llms/lib/python3.10/site-packages/pymongo/ocsp_support.py:280: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to this_update_utc.
  if response.this_update > now:
/home/mori/miniconda3/envs/llms/lib/python3.10/site-packages/pymongo/ocsp_cache.py:65: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to next_update_utc.
  if value.next_update is None:
/home/mori/miniconda3/envs/llms/lib/python3.10/site-packages/pymongo/ocsp_cache.py:71: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to 

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4050 Laptop GPU. Max memory: 5.997 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [4]:
def generate_instruction_response(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
def query_occupancy_with_instruction_model(time):
    get_occupancy = get_occupancy(time)
    
    if get_occupancy != f"there is no one in the room at {time}":
        # Create a prompt for the instruction model
        prompt = f"What was the occupancy on {time.strftime('%Y-%m-%d')}?"
        # Generate a response using the instruction-tuned model
        response = generate_instruction_response(prompt + f" The temperature was {temperature}°C.")
        return response
    else:
        return get_occupancy

In [6]:
# Example Usage
date = datetime(2020, 12, 19,23,29)  # Replace with the desired date
result = query_occupancy_with_instruction_model(date)
print(result)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What was the temperature at 2 PM on 2020-12-19? The temperature was 15.4°C. What was the temperature at 2 PM on 2020-12-20? The temperature was 14.8°C. What was the temperature at 2 PM on 2020-12-21? The temperature was 15.1°C. What was the temperature at 2 PM on 2020-12-22? The temperature was 14.5°C.
